# Basic Vector Search from Scratch

For this exercise we will implement basic vector search
from scratch with just numpy.<br/>
This will give us a feel
for what's happening under the hood in vector databases.

In [ ]:
!pip install numpy pytest

## Euclidean distance

There are many ways to measure the distance between two vectors.
Let's write a function that computes the `Euclidean distance` 
between vectors. 

This function should take as input two vectors and return
the euclidean distance between them.

For more details you can read this [kaggle page](https://www.kaggle.com/code/paulrohan2020/euclidean-distance-and-normalizing-a-vector)


In [1]:
import numpy as np

In [2]:
def euclidean_distance(v1: np.ndarray, v2: np.ndarray) -> float:
    """
    Compute the Euclidean distance between two vectors.

    Parameters
    ----------
    v1 : np.ndarray
        First vector.
    v2 : np.ndarray
        Second vector.

    Returns
    -------
    float
        Euclidean distance between `v1` and `v2`.
    """
    # Verifica se os formatos dos dois vetores são diferentes.
    if v1.shape != v2.shape:
        # Se os formatos não forem iguais, levanta um erro ValueError,
        # indicando que os vetores devem ter o mesmo formato para o cálculo da distância.
        raise ValueError("Vectors must have the same shape.")
    else:
        # Se os formatos forem iguais, calcula a diferença elemento a elemento entre os vetores.
        dist = v1 - v2
    # Calcula a norma (magnitude) euclidiana do vetor de diferença ('dist').
    # O argumento 'axis=len(dist.shape)-1' garante que a norma seja calculada ao longo do último eixo
    # o que é apropriado para vetores, mesmo que eles estejam dentro de arrays multidimensionais.
    return np.linalg.norm(dist, axis=len(dist.shape)-1)

In [3]:
import pytest

In [4]:
# Define o primeiro vetor NumPy.
v1 = np.array([1, 2, 3])

# Define o segundo vetor NumPy.
v2 = np.array([4, 5, 6])

# Calcula a distância euclidiana manualmente:
# 1. Subtrai os elementos correspondentes dos dois vetores (v1 - v2).
# 2. Eleva ao quadrado cada elemento do vetor resultante (** 2).
# 3. Soma todos os elementos do vetor resultante (np.sum(...)).
# 4. Calcula a raiz quadrada da soma (np.sqrt(...)), que é a distância euclidiana.
dist = np.sqrt(np.sum((v1 - v2) ** 2))

# Utiliza a função 'euclidean_distance' definida anteriormente para calcular a distância entre os mesmos vetores.
# Em seguida, usa a instrução 'assert' para verificar se o resultado da função
# é igual ao valor da distância calculada manualmente. Se os valores forem diferentes,
# um AssertionError será levantado, indicando que a função não está funcionando como esperado.
assert euclidean_distance(v1, v2) == dist

## KNN search

Using the distance function you just wrote, write a function that 
finds the k-nearest neighbors of a query vector.

This function should take as input a query vector, a 2d array of database vectors,
and an integer k the number of nearest neighbors to return. And it should return 
the vectors that are the k-nearest neighbors of the query vector.


In [5]:
def find_nearest_neighbors(query: np.ndarray,
                           vectors: np.ndarray,
                           k: int = 1) -> np.ndarray:
    """
    Find k-nearest neighbors of a query vector.

    Parameters
    ----------
    query : np.ndarray
        Query vector.
    vectors : np.ndarray
        Vectors to search.
    k : int, optional
        Number of nearest neighbors to return, by default 1.

    Returns
    -------
    np.ndarray
        The `k` nearest neighbors of `query` in `vectors`.
    """
    # Verifica se o valor de k é menor que 1.
    if k < 1:
        # Se k for inválido, levanta um erro ValueError com uma mensagem apropriada.
        raise ValueError("k must be at least 1.")
    # Verifica se o valor de k é maior que o número total de vetores disponíveis.
    if k > vectors.shape[0]:
        # Se k exceder o número de vetores, levanta um ValueError.
        raise ValueError("k must not exceed the number of vectors.")
    # Calcula a distância euclidiana entre o vetor de consulta e cada vetor na matriz 'vectors'.
    # Isso é feito usando uma list comprehension que aplica a função 'euclidean_distance'
    # a cada vetor em 'vectors' e converte a lista resultante de distâncias em um array NumPy.
    distances = np.array([euclidean_distance(query, v) for v in vectors])
    # Encontra os índices dos 'k' menores valores no array de 'distances'.
    # 'np.argsort(distances)' retorna os índices que ordenariam o array 'distances' em ordem crescente.
    # Ao fatiar o resultado com '[:k]', pegamos os primeiros 'k' índices, que correspondem às menores distâncias.
    nearest_indices = np.argsort(distances)[:k]
    # Usa os 'nearest_indices' para indexar a matriz 'vectors' e retornar os vetores correspondentes.
    # Isso efetivamente retorna os 'k' vizinhos mais próximos do vetor de consulta.
    return vectors[nearest_indices]

In [6]:
# Cria uma matriz NumPy 'mat' com 500 linhas e 64 colunas, preenchida com números aleatórios
# seguindo uma distribuição normal padrão. Cada linha desta matriz pode ser interpretada como um vetor.
mat = np.random.randn(500, 64)

# Cria um vetor de consulta 'query' com 64 elementos, também preenchido com números aleatórios
# de uma distribuição normal padrão. Este é o vetor para o qual queremos encontrar os vizinhos mais próximos.
query = np.random.randn(64)

# Define o número de vizinhos mais próximos que queremos encontrar (k=8).
k = 8

# Calcula a norma (magnitude) de cada vetor (linha) na matriz 'mat' ao longo do eixo 1 (as colunas).
# Este cálculo não é diretamente usado para encontrar os vizinhos mais próximos neste trecho,
# mas pode ser útil em outros contextos para análise ou normalização de vetores.
norms = np.linalg.norm(mat, axis=1)

# Calcula a distância euclidiana entre o vetor de consulta 'query' e cada vetor (linha) na matriz 'mat'.
# O resultado 'expected' é um array 1D contendo as distâncias.
expected_distances = np.linalg.norm(mat - query, axis=1)

# Encontra os índices dos 'k' menores valores no array de distâncias 'expected_distances' usando 'np.argsort'.
# Em seguida, usa esses índices para selecionar as 'k' linhas correspondentes da matriz 'mat'.
# 'expected' agora contém os 'k' vizinhos mais próximos de 'query' em 'mat', calculados diretamente.
expected = mat[np.argsort(expected_distances)[:k], :]

# Chama a função 'find_nearest_neighbors' (que presumimos estar definida anteriormente)
# para encontrar os 'k' vizinhos mais próximos de 'query' dentro da matriz 'mat'.
# O resultado é armazenado na variável 'actual'.
actual = find_nearest_neighbors(query, mat, k=k)

# Utiliza a função 'np.allclose' para comparar se a matriz de vizinhos encontrados pela nossa função ('actual')
# é aproximadamente igual à matriz de vizinhos esperados ('expected'). 'np.allclose' é preferível a '==' para
# comparar arrays de ponto flutuante, pois leva em consideração pequenas diferenças devido à precisão numérica.
# Se os arrays não forem aproximadamente iguais, um AssertionError será levantado, indicando uma possível falha
# na implementação da função 'find_nearest_neighbors'.
assert np.allclose(actual, expected)

## Other distance metrics

For this problem we'll write a new distance function and modify 
our nearest neighbors function to accept a distance metric.


Write a function that computes the [cosine distance](https://en.wikipedia.org/wiki/Cosine_similarity) between vectors.

In [7]:
from typing import Union

def cosine_distance(v1: np.ndarray, v2: np.ndarray) -> Union[float, np.ndarray]:
    """
    Compute the cosine distance between two vectors.

    Parameters
    ----------
    v1 : np.ndarray
        First vector.
    v2 : np.ndarray
        Second vector.

    Returns
    -------
    float
        Cosine distance between `v1` and `v2`.
    """
    # Verifica se os formatos dos dois vetores são diferentes.
    if v1.shape != v2.shape:
        # Se os formatos não forem iguais, levanta um erro ValueError,
        # indicando que os vetores devem ter o mesmo formato para o cálculo da distância cosseno.
        raise ValueError("Vectors must have the same shape.")

    # Calcula o produto escalar (dot product) entre os dois vetores.
    dot_product = np.dot(v1, v2)

    # Calcula a norma (magnitude) do primeiro vetor.
    norm_v1 = np.linalg.norm(v1)

    # Calcula a norma (magnitude) do segundo vetor.
    norm_v2 = np.linalg.norm(v2)

    # Verifica se alguma das normas é zero. Se for, isso significa que um dos vetores é o vetor nulo,
    # e a distância cosseno não está bem definida nesse caso.
    if norm_v1 == 0 or norm_v2 == 0:
        raise ValueError("One of the vectors is zero.")

    # Calcula a similaridade cosseno dividindo o produto escalar pelo produto das normas.
    # A distância cosseno é então calculada como 1 menos a similaridade cosseno.
    return 1 - (dot_product / (norm_v1 * norm_v2))

**HINT** Please make sure you understand the difference between cosine similarity and cosine distance

Now, rewrite the `find_nearest_neighbors` function to accept a distance metric so you can use either Euclidean or Cosine distance

In [8]:
def find_nearest_neighbors(query: np.ndarray,
                           vectors: np.ndarray,
                           k: int = 1,
                           distance_metric="euclidean") -> np.ndarray:
    """
    Find k-nearest neighbors of a query vector with a configurable
    distance metric.

    Parameters
    ----------
    query : np.ndarray
        Query vector.
    vectors : np.ndarray
        Vectors to search.
    k : int, optional
        Number of nearest neighbors to return, by default 1.
    distance_metric : str, optional
        Distance metric to use, by default "euclidean".

    Returns
    -------
    np.ndarray
        The `k` nearest neighbors of `query` in `vectors`.
    """
    # Verifica se o valor de k é menor que 1.
    if k < 1:
        # Se k for inválido, levanta um erro ValueError com uma mensagem apropriada.
        raise ValueError("k must be at least 1.")
    # Verifica se o valor de k é maior que o número total de vetores na matriz 'vectors'.
    if k > vectors.shape[0]:
        # Se k exceder o número de vetores disponíveis, levanta um ValueError.
        raise ValueError("k must not exceed the number of vectors.")

    # Calcula as distâncias entre o vetor de consulta e todos os vetores na matriz 'vectors'
    # com base na métrica de distância especificada.
    if distance_metric == "euclidean":
        distances = np.array([euclidean_distance(query, v) for v in vectors])
    elif distance_metric == "cosine":
        distances = np.array([cosine_distance(query, v) for v in vectors])
    else:
        # Se a métrica de distância fornecida não for "euclidean" nem "cosine",
        # levanta um ValueError indicando que a métrica é desconhecida.
        raise ValueError(f"Unknown distance metric: {distance_metric}")

    # Encontra os índices dos 'k' menores valores no array de 'distances'.
    # 'np.argsort(distances)' retorna os índices que ordenariam o array 'distances' em ordem crescente.
    # Ao fatiar o resultado com '[:k]', pegamos os primeiros 'k' índices, que correspondem às menores distâncias.
    nearest_indices = np.argsort(distances)[:k]

    # Usa os 'nearest_indices' para selecionar as 'k' linhas correspondentes da matriz 'vectors'.
    # O ':,' garante que todas as colunas dessas linhas sejam retornadas, obtendo assim os vetores vizinhos mais próximos.
    return vectors[nearest_indices, :]

## Exploration

Now that we have a nearest neighbors function that accepts a distance metric, <br/>
let's explore the differences between Euclidean distance and cosine distance.

Would you expect same or different answers?

In [9]:
# You might find this function useful

def generate_vectors(num_vectors: int, num_dim: int,
                     normalize: bool = True) -> np.ndarray:
    """
    Generate random embedding vectors.

    Parameters
    ----------
    num_vectors : int
        Number of vectors to generate.
    num_dim : int
        Dimensionality of the vectors.
    normalize : bool, optional
        Whether to normalize the vectors, by default True.

    Returns
    -------
    np.ndarray
        Randomly generated `num_vectors` vectors with `num_dim` dimensions.
    """
    # Gera uma matriz de números aleatórios entre 0 e 1 com a forma (num_vectors, num_dim).
    # 'np.random.rand' é usado aqui para criar vetores com componentes não negativos.
    vectors = np.random.rand(num_vectors, num_dim)

    # Verifica se a normalização foi solicitada.
    if normalize:
        # Normaliza cada vetor (linha) da matriz 'vectors' para ter norma unitária.
        # 'np.linalg.norm(vectors, axis=1, keepdims=True)' calcula a norma L2 de cada linha (axis=1)
        # e 'keepdims=True' garante que o resultado tenha a forma (num_vectors, 1), o que permite
        # a divisão por broadcasting para normalizar cada vetor individualmente.
        vectors /= np.linalg.norm(vectors, axis=1, keepdims=True)

    # Retorna a matriz de vetores gerados (normalizados ou não).
    return vectors

In [10]:
# Gere novos vetores aleatórios para esta célula de exploração
num_vectors_exploracao = 10
num_dim_exploracao = 5
query_vector_exploracao = np.random.rand(num_dim_exploracao)
vectors_exploracao = generate_vectors(num_vectors_exploracao, num_dim_exploracao)

# Encontre os vizinhos mais próximos usando distância Euclidiana
euclidean_neighbors_exploracao = find_nearest_neighbors(query_vector_exploracao, vectors_exploracao, k=3, distance_metric="euclidean")

# Encontre os vizinhos mais próximos usando distância Cosseno
cosine_neighbors_exploracao = find_nearest_neighbors(query_vector_exploracao, vectors_exploracao, k=3, distance_metric="cosine")

print("Query Vector:", query_vector_exploracao)
print("Generated Vectors:\n", vectors_exploracao)
print("\nNearest Neighbors (Euclidean):\n", euclidean_neighbors_exploracao)
print("\nNearest Neighbors (Cosine):\n", cosine_neighbors_exploracao)

Query Vector: [0.08640712 0.01136141 0.8832798  0.69735563 0.55377673]
Generated Vectors:
 [[0.22775788 0.29205953 0.62153262 0.58466383 0.36700544]
 [0.02744186 0.55774124 0.33578092 0.52172687 0.5506577 ]
 [0.43256683 0.26864639 0.17493958 0.79961558 0.26594383]
 [0.63256264 0.38420158 0.03237274 0.61888217 0.26113313]
 [0.30099179 0.76297524 0.27414142 0.48399752 0.13366229]
 [0.63657681 0.55674913 0.22005364 0.35618258 0.33092407]
 [0.16066516 0.57716193 0.0589824  0.65957851 0.45005342]
 [0.4024446  0.34612413 0.15057285 0.48716466 0.67693046]
 [0.09005301 0.56392383 0.39812155 0.48068898 0.53321451]
 [0.19708636 0.05582402 0.04426748 0.62134264 0.75499296]]

Nearest Neighbors (Euclidean):
 [[0.22775788 0.29205953 0.62153262 0.58466383 0.36700544]
 [0.09005301 0.56392383 0.39812155 0.48068898 0.53321451]
 [0.02744186 0.55774124 0.33578092 0.52172687 0.5506577 ]]

Nearest Neighbors (Cosine):
 [[0.22775788 0.29205953 0.62153262 0.58466383 0.36700544]
 [0.09005301 0.56392383 0.398121